# Transformer 기반 법률 문서 요약 모델 학습 및 결과 분석

## 환경 세팅

In [2]:
# 필요한 라이브러리 설치
# 로컬 환경에서는 이미 설치되어 있다면 이 셀을 건너뛰어도 됩니다.
try:
    import transformers
    print("Transformers library is already installed.")
except ImportError:
    !pip install -q transformers datasets rouge-score accelerate
    print("Installed necessary libraries.")

# 한글 폰트 설치 (Colab 환경에서 시각화 필요 시 사용, 로컬은 생략 가능)
try:
    import google.colab
    !sudo apt-get install -y fonts-nanum
    !sudo fc-cache -fv
    !rm ~/.cache/matplotlib -rf
    print("Installed Korean fonts for Colab.")
except ImportError:
    pass

Transformers library is already installed.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,422 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(

In [3]:
import os
import sys
import logging
import warnings

warnings.filterwarnings("ignore")

def setup_logger():
    # '__main__' 로거를 가져옵니다.
    logger = logging.getLogger(__name__)

    # 1. 기존 핸들러가 있다면 모두 제거 (셀 재실행 시 중복 방지)
    if logger.hasHandlers():
        logger.handlers.clear()

    logger.setLevel(logging.INFO)

    # 2. 부모 로거로 로그를 전달하지 않도록 설정 (중복 출력 방지 핵심)
    logger.propagate = False

    # 3. 새로운 핸들러 및 포맷 설정
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    stream_handler = logging.StreamHandler(sys.stdout)
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    return logger

# 로거 초기화
logger = setup_logger()
logger.info("Logger setup completed.")

2026-01-21 04:10:15,071 - INFO - Logger setup completed.


In [4]:
# 프로젝트 및 데이터 경로 설정
PROJECT_NAME = "12_NLP_Summarization" # 프로젝트 폴더명 (필요에 따라 수정)

try:
    # Google Colab 환경 확인
    from google.colab import drive
    drive.mount('/content/drive')

    # Colab 경로 설정 (Google Drive 내의 경로)
    # 예: /content/drive/MyDrive/Projects/Document_Summarization_Mission
    BASE_DIR = os.path.join("/content/drive/MyDrive/codeit", PROJECT_NAME)
    logger.info("Environment: Google Colab")

except ImportError:
    # 로컬 환경 설정
    # 현재 작업 디렉토리를 기준으로 설정
    BASE_DIR = os.path.abspath(f"./{PROJECT_NAME}")
    logger.info("Environment: Local")

# 데이터 및 결과 저장 디렉토리 정의
DATA_DIR = os.path.join(BASE_DIR, "data")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")

# 디렉토리가 존재하지 않으면 생성
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

logger.info(f"Base Directory: {BASE_DIR}")
logger.info(f"Data Directory: {DATA_DIR}")
logger.info(f"Output Directory: {OUTPUT_DIR}")

Mounted at /content/drive
2026-01-21 04:10:40,502 - INFO - Environment: Google Colab
2026-01-21 04:10:42,884 - INFO - Base Directory: /content/drive/MyDrive/codeit/12_NLP_Summarization
2026-01-21 04:10:42,885 - INFO - Data Directory: /content/drive/MyDrive/codeit/12_NLP_Summarization/data
2026-01-21 04:10:42,886 - INFO - Output Directory: /content/drive/MyDrive/codeit/12_NLP_Summarization/output


In [5]:
import torch

def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        logger.info(f"Device: GPU ({torch.cuda.get_device_name(0)})")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        logger.info("Device: MPS (Apple Silicon)")
    else:
        device = torch.device("cpu")
        logger.info("Device: CPU")
    return device

device = get_device()

# 시드 고정 함수 (재현성을 위해 설정)
def seed_everything(seed):
    import random
    import numpy as np

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

SEED = 42
seed_everything(SEED)
logger.info(f"Random seed set to {SEED}")

2026-01-21 04:14:54,819 - INFO - Device: CPU
2026-01-21 04:14:54,850 - INFO - Random seed set to 42


## EDA

In [7]:
import os
import glob
import json
import pandas as pd

# 1. 전체 데이터 파일 스캔
def scan_data_files(data_dir):
    json_files = glob.glob(os.path.join(data_dir, "*.json"))
    # 하위 폴더가 있을 경우를 대비해 recursive 탐색이 필요하면 아래 주석 해제
    # json_files = glob.glob(os.path.join(data_dir, "**", "*.json"), recursive=True)

    logger.info(f"검색된 JSON 파일 개수: {len(json_files)}")

    file_info = []
    for f in json_files:
        size_mb = os.path.getsize(f) / (1024 * 1024)
        file_info.append({
            "File Name": os.path.basename(f),
            "Path": f,
            "Size (MB)": round(size_mb, 2)
        })

    df_files = pd.DataFrame(file_info)
    return df_files

# 파일 목록 출력
df_files = scan_data_files(DATA_DIR)
display(df_files)


2026-01-21 04:15:12,122 - INFO - 검색된 JSON 파일 개수: 6


,File Name,Path,Size (MB)
0,train_original_editorial.json,/content/drive/MyDrive/codeit/12_NLP_Summariza...,313.23
1,valid_original_editorial.json,/content/drive/MyDrive/codeit/12_NLP_Summariza...,35.33
2,valid_original_law.json,/content/drive/MyDrive/codeit/12_NLP_Summariza...,8.51
3,valid_original_news.json,/content/drive/MyDrive/codeit/12_NLP_Summariza...,139.97
4,train_original_law.json,/content/drive/MyDrive/codeit/12_NLP_Summariza...,82.42
5,train_original_news.json,/content/drive/MyDrive/codeit/12_NLP_Summariza...,1108.73


> 법률 문서로 타깃하여 프로젝트 진행
- 과제 일정이 딜레이되어있고, 저번 과제에서 리소스 부족을 체감 함
- 데이터의 크기는 3종의 데이터 중에서 가장 작지만, 전문 용어가 많고 문장 구조가 복잡하여 난이도는 높은 편일 것이라고 예측하여 선택

In [8]:
# -------------------------------------------------------------------------
# 법률 데이터 파일 경로 설정
# -------------------------------------------------------------------------

try:
    # 'File Name' 컬럼에서 'law'가 포함된 행들을 필터링합니다.
    law_files = df_files[df_files['File Name'].str.contains('law', case=False)]

    # 그 중 학습용(train)과 검증용(valid)을 각각 추출합니다.
    train_file_path = law_files[law_files['File Name'].str.contains('train', case=False)]['Path'].values[0]
    valid_file_path = law_files[law_files['File Name'].str.contains('valid', case=False)]['Path'].values[0]

    # 분석 타겟은 일단 학습용(train)으로 설정합니다.
    target_file_path = train_file_path

    print(f"선택된 학습 데이터: {os.path.basename(train_file_path)}")
    print(f"선택된 검증 데이터: {os.path.basename(valid_file_path)}")

except IndexError:
    print("\n경고: 파일을 찾지 못했습니다. 파일 목록을 다시 확인해 주세요.")
    # 수동 지정 예시:
    # target_file_path = "/content/drive/MyDrive/.../train_original_law.json"

선택된 학습 데이터: train_original_law.json
선택된 검증 데이터: valid_original_law.json


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

def load_and_analyze_legal_data(file_path):
    logger.info(f"데이터 로딩 중... ({file_path})")

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # AI Hub 데이터 구조는 보통 {'documents': [...] } 형태입니다.
    # 실제 키 값을 확인 후 수정이 필요할 수 있습니다.
    if isinstance(data, dict) and 'documents' in data:
        documents = data['documents']
    elif isinstance(data, list):
        documents = data
    else:
        logger.error("JSON 구조가 예상과 다릅니다. 확인이 필요합니다.")
        raise ValueError("JSON 구조가 예상과 다릅니다. 확인이 필요합니다.")

    logger.info(f"총 문서 개수: {len(documents)}")

    # 데이터 파싱 (필요한 컬럼만 추출)
    # 법률 문서는 보통 'text'(본문)와 'abstractive'(생성요약) 키를 가집니다.
    rows = []
    for doc in documents:
        # 본문이 문장 리스트로 되어 있는 경우 합치기 (AI Hub 일반적 구조)
        # text = [[sentence dict, ...], [sentence dict, ...]] 형태일 수 있음
        try:
            # 구조에 따라 파싱 로직이 달라질 수 있음. 가장 일반적인 구조 가정:
            # text 리스트 안의 문장들을 공백으로 join
            full_text = " ".join([sent['sentence'] for para in doc['text'] for sent in para])
            summary = doc['abstractive'][0] # 요약문

            rows.append({
                'id': doc['id'],
                'category': doc.get('category', 'law'),
                'source_text': full_text,
                'target_text': summary
            })
        except Exception as e:
            # 데이터 구조가 다를 경우 에러 로그
            # print(f"Error parsing doc {doc.get('id')}: {e}")
            continue

    df = pd.DataFrame(rows)

    # 길이 분석 (글자 수 및 단어 수)
    df['source_len'] = df['source_text'].apply(len)
    df['target_len'] = df['target_text'].apply(len)

    return df

# 데이터 로드 실행
df_law = load_and_analyze_legal_data(target_file_path)

# 판다스 출력 설정 변경
pd.set_option('display.max_colwidth', None)  # 컬럼 너비 제한 해제
pd.set_option('display.max_rows', 10)        # 출력 행 수 제한 (너무 많으면 브라우저가 느려짐)

# 상위 3개 샘플 출력
print("\n[데이터 샘플 확인]")
display(df_law.head(3))

# 통계 정보 출력
print("\n[길이 통계 정보]")
display(df_law[['source_len', 'target_len']].describe().round(1))

2026-01-21 04:15:26,102 - INFO - 데이터 로딩 중... (/content/drive/MyDrive/codeit/12_NLP_Summarization/data/train_original_law.json)
2026-01-21 04:15:29,890 - INFO - 총 문서 개수: 24329

[데이터 샘플 확인]


,id,category,source_text,target_text,source_len,target_len
0,100004,일반행정,"원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.",원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.,372,97
1,100005,세무,"수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 회사의 직수출실적을 지원하기 위하여 동 회사와 수출대행계약을 체결하고 동 회사를 수익자로 하여 신용장을 개설한 후 동 회사명의로 제품을 수출하면서 그 수출물품에 관하여 소외 회사는 원고로부터 세금계산서를 교부받지 않았을 뿐 아니라, 부가가치세신고시 이를 수출로 신고하지 않았고, 외화수입금도 원신용장상 수익자인 소외 회사에 입금되나 당해은행에서 매수 즉시, 원화로 원고의 당좌계정에 입금되고 소외 회사는 대행수수료만 받아왔다면 소외 회사는 수출에 실지 참여한 바 없이 원고의 수출대행을 한 자로 명목상의 수출자에 불과할 뿐 원고가 이를 실질적으로 수출하여 그 대금에 해당하는 외화를 수입하였다 할 것이므로, 원고는 구 법인세법시행령(1983.12.31 대통령령 제10119호로 개정되기 전의 령) 제17조 제1항 제1호 소정의 수출사업을 영위하는 법인에 해당되어 그 수출금액에 대응한 해외시장개척준비금을 설정할 수 있다.","수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기 위하여 동 회사와 수출대행계약을 체결하고, 소외 회사는 수출에 실지 참여한 바 없이 명목상의 수출자에 불과할 뿐 원고가 이를 실질적으로 수출하여 그 대금에 해당하는 외화를 수입하였다 할 것이므로, 원고는 구 법인세법시행령(1983.12.31 대통령령 제10119호로 개정되기 전의 령) 제17조 제1항 제1호 소정의 수출사업을 영위하는 법인에 해당되어 그 수출금액에 대응한 해외시장개척준비금을 설정할 수 있다.",505,276
2,100006,세무,"가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후 다시 제3자에게 매매계약을 원인으로 한 가등기를 경료해준 일이 있다 하더라도 아직 채권자와의 사이에 정산절차를 밟지 아니하였고, 위 부동산의 본등기 당시의 시가가 원리금 채무의 2배를 넘고 있었다면 위 부동산은 채무담보로 제공되어 아직 그 채무의 변제에 충당되지 아니한 상태에 있는 경우라 할 것이고, 따라서 양도행위가 이루어진 바 없다고 할 것이다.",가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 다시 제3자에게 매매계약을 원인으로 한 가등기를 경료해주었더라도 아직 채권자와의 사이에 정산절차를 밟지 않았고 위 부동산은 본등기 당시의 시가가 원리금 채무의 2배를 넘고 있었다면 위 부동산은 채무담보로 제공되어 아직 그 채무의 변제에 충당되지 않은 상태에 있으므로 양도행위가 이루어진 바 없다고 보아야 한다.,243,219



[길이 통계 정보]


,source_len,target_len
count,24329.0,24329.0
mean,669.9,198.8
std,449.0,78.3
min,67.0,33.0
25%,360.0,144.0
50%,539.0,188.0
75%,844.0,240.0
max,3993.0,1000.0


- 약 24300 건의 문서는 학습량으로 충분할 것으로 보임
- 문장 하나하나가 수식이 많고 긴 만연체
- 긴 법률 전문 용어가 많아, 일반적인 토크나이저는 이를 잘게 쪼개어 의미를 희석시킬 수 있음을 고려해야 함


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_length_distribution_interactive(df):
    # 서브플롯 생성 (1행 2열)
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Source Text Length Distribution', 'Target Summary Length Distribution'),
        horizontal_spacing=0.1
    )

    # 1. 본문 길이 분포 (Histogram)
    fig.add_trace(
        go.Histogram(
            x=df['source_len'],
            name='Source',
            nbinsx=50,
            marker_color='skyblue',
            hovertemplate='Length: %{x}<br>Count: %{y}'
        ),
        row=1, col=1
    )

    # 2. 요약문 길이 분포 (Histogram)
    fig.add_trace(
        go.Histogram(
            x=df['target_len'],
            name='Target',
            nbinsx=50,
            marker_color='orange',
            hovertemplate='Length: %{x}<br>Count: %{y}'
        ),
        row=1, col=2
    )

    # 평균선 추가 (Source)
    source_mean = df['source_len'].mean()
    fig.add_vline(
        x=source_mean,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mean: {source_mean:.0f}",
        row=1, col=1
    )

    # 평균선 추가 (Target)
    target_mean = df['target_len'].mean()
    fig.add_vline(
        x=target_mean,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mean: {target_mean:.0f}",
        row=1, col=2
    )

    # 레이아웃 업데이트
    fig.update_layout(
        title_text="Document Length Analysis (Character Count)",
        title_x=0.5,
        showlegend=False,
        template="plotly_white",
        height=500
    )

    fig.update_xaxes(title_text="Length", row=1, col=1)
    fig.update_xaxes(title_text="Length", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=2)

    fig.show()

def print_length_statistics(df):
    for col, label in zip(['source_len', 'target_len'], ['Source (본문)', 'Target (요약)']):
        stats = df[col].describe()
        p90 = df[col].quantile(0.90)
        p95 = df[col].quantile(0.95)
        p99 = df[col].quantile(0.99)

        print(f"--- {label} 길이 통계 ---")
        print(f"평균(Mean): {stats['mean']:.1f}자")
        print(f"중앙값(Median): {stats['50%']:.1f}자")
        print(f"최대값(Max): {stats['max']:.1f}자")
        print(f"90th Percentile: {p90:.1f}자 (90% 데이터가 이 길이 이하)")
        print(f"95th Percentile: {p95:.1f}자 (95% 데이터가 이 길이 이하)")
        print(f"99th Percentile: {p99:.1f}자 (99% 데이터가 이 길이 이하)")
        print()


# 그래프 실행
plot_length_distribution_interactive(df_law)
print_length_statistics(df_law)

--- Source (본문) 길이 통계 ---
평균(Mean): 669.9자
중앙값(Median): 539.0자
최대값(Max): 3993.0자
90th Percentile: 1234.0자 (90% 데이터가 이 길이 이하)
95th Percentile: 1557.0자 (95% 데이터가 이 길이 이하)
99th Percentile: 2314.6자 (99% 데이터가 이 길이 이하)

--- Target (요약) 길이 통계 ---
평균(Mean): 198.8자
중앙값(Median): 188.0자
최대값(Max): 1000.0자
90th Percentile: 300.0자 (90% 데이터가 이 길이 이하)
95th Percentile: 346.0자 (95% 데이터가 이 길이 이하)
99th Percentile: 428.7자 (99% 데이터가 이 길이 이하)



- 평균이 중앙값보다 큰 long-tail 분포
- 본문의 95% 지점이 1,557자로 최소 1024 토큰이상을 처리할 수 있는 설정이나 모델이 필요할 것
- 또한, 법률 문서는 앞부분(사건 개요)와 뒷부분(법원의 최종 판단)이 모두 중요하므로 양 끝을 합치는 방식을 활용 예정
- 요약문의 99%가 428자 이내로 생성 모델의 max_target_length는 512자로 설정하여 커버할 예정

## 토크나이저 로드

In [10]:
# 1. 필요한 라이브러리 설치 (필요시)
# !pip install -q transformers sentencepiece

from transformers import PreTrainedTokenizerFast

# 한국어 요약에 성능이 검증된 KoBART 토크나이저 사용
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [14]:
def check_token_stats_full(df, tokenizer):
    logger.info(f"전체 데이터({len(df)}건) 토큰 길이 분석 시작... 잠시만 기다려주세요.")

    # 전체 텍스트에 대해 토큰화 수행 (추론만 하므로 가속을 위해 padding/truncation은 제외)
    # 문장 리스트를 바로 encode에 넣으면 시간이 걸리므로 리스트 컴프리헨션 사용
    tokenized_lens = [len(tokenizer.encode(t)) for t in df['source_text']]
    df['source_token_len'] = tokenized_lens

    import numpy as np
    p50 = np.percentile(tokenized_lens, 50)
    p90 = np.percentile(tokenized_lens, 90)
    p95 = np.percentile(tokenized_lens, 95)
    p99 = np.percentile(tokenized_lens, 99)

    logger.info(f"--- 전체 데이터 토큰 통계 ---")
    logger.info(f"평균: {np.mean(tokenized_lens):.1f} / 중앙값: {p50:.1f}")
    logger.info(f"최대값: {np.max(tokenized_lens)}")
    logger.info(f"90% 지점: {p90:.1f}")
    logger.info(f"95% 지점: {p95:.1f}")
    logger.info(f"99% 지점: {p99:.1f}")

    return tokenized_lens

# 전체 분석 실행
source_token_lens = check_token_stats_full(df_law, tokenizer)

2026-01-21 04:17:35,250 - INFO - 전체 데이터(24329건) 토큰 길이 분석 시작... 잠시만 기다려주세요.
2026-01-21 04:17:58,796 - INFO - --- 전체 데이터 토큰 통계 ---
2026-01-21 04:17:58,800 - INFO - 평균: 296.8 / 중앙값: 241.0
2026-01-21 04:17:58,805 - INFO - 최대값: 1748
2026-01-21 04:17:58,806 - INFO - 90% 지점: 543.0
2026-01-21 04:17:58,807 - INFO - 95% 지점: 685.0
2026-01-21 04:17:58,807 - INFO - 99% 지점: 1010.4


- max_input_len = 1024 로 99% 커버 가능함
- 최대값 1748이 존재하지만, 극소수이고 (head/tail) 보존하는 윈도우 슬라이싱으로 커버 할 예정

## 전처리 보조 함수 정의

In [11]:
def advanced_window_slicing(text, tokenizer, max_len=1024, head_ratio=0.7):
    """
    긴 법률 문서를 처리하기 위해 Head(앞부분)와 Tail(뒷부분)을 추출하여 결합하는 함수입니다.
    특수 토큰(BOS, EOS)의 위치를 정확하게 제어하여 모델의 혼란을 방지합니다.

    Args:
        text (str): 원본 문자열 (법률 본문)
        tokenizer (PreTrainedTokenizer): 사용할 토크나이저 객체 (예: KoBART)
        max_len (int): 모델에 입력할 최대 토큰 길이 (Default: 1024)
        head_ratio (float): 전체 길이 중 앞부분(사건 개요 등)이 차지할 비율 (0.0 ~ 1.0)
                            나머지(1.0 - head_ratio)는 뒷부분(판결 결과 등)에 할당됩니다.

    Returns:
        list[int]: [BOS] + Head Tokens + Tail Tokens + [EOS] 형태로 구성된 토큰 ID 리스트
    """

    # 1. 순수 텍스트만 토큰화 (특수 토큰 자동 추가 방지)
    # add_special_tokens=False를 해야 나중에 우리가 원하는 위치에만 BOS/EOS를 붙일 수 있습니다.
    input_ids = tokenizer.encode(text, add_special_tokens=False)
    total_len = len(input_ids)

    # 2. 모델이 요구하는 특수 토큰 개수 파악
    # KoBART 등 대부분의 모델은 앞뒤로 <s>, </s>가 붙으므로 보통 2개 공간이 필요합니다.
    # tokenizer.num_special_tokens_to_add() 메서드가 있다면 그걸 써도 되지만,
    # 안전하게 2로 가정하거나 build_inputs 로직에 맡깁니다.
    # 여기서는 안전 마진으로 2를 뺍니다.
    effective_max_len = max_len - 2

    # 3. 길이가 충분히 짧다면 자를 필요 없음 (바로 특수 토큰 붙여서 반환)
    if total_len <= effective_max_len:
        return tokenizer.build_inputs_with_special_tokens(input_ids)

    # 4. Head/Tail 길이 계산
    # 예: max_len=1024 -> effective=1022 -> head=715, tail=307
    head_len = int(effective_max_len * head_ratio)
    tail_len = effective_max_len - head_len

    # 5. 슬라이싱 (Slicing)
    # 처음부터 head_len까지, 그리고 끝에서 tail_len부터 끝까지 가져옴
    head_tokens = input_ids[:head_len]
    tail_tokens = input_ids[-tail_len:]

    # 6. 결합 및 특수 토큰 부착
    # [Head] + [Tail] 상태의 리스트를 만든 뒤,
    # build_inputs_with_special_tokens를 호출하면 앞뒤에 정확히 <s>, </s>를 붙여줍니다.
    combined_ids = head_tokens + tail_tokens
    final_input_ids = tokenizer.build_inputs_with_special_tokens(combined_ids)

    # [디버깅용 주석]
    # 실제 학습 시에는 주석 처리하거나 로거 레벨을 확인하세요.
    # logger.debug(f"Original: {total_len} -> Sliced: {len(final_input_ids)} (Head: {head_len}, Tail: {tail_len})")

    return final_input_ids

## 데이터셋 클래스 정의 및 데이터 로더 구축

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

class LegalSummaryDataset(Dataset):
    """
    법률 문서 요약을 위한 PyTorch Dataset 클래스
    """
    def __init__(self, df, tokenizer, max_input_len=1024, max_target_len=256):
        self.data = df
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # 1. 본문 처리 (Advanced Window Slicing 적용)
        # 95% 커버리지를 위해 max_input_len 내에서 Head/Tail 보존
        input_ids = advanced_window_slicing(
            row['source_text'],
            self.tokenizer,
            max_len=self.max_input_len,
            head_ratio=0.7 # 사실관계(Head) 비중을 높게 설정
        )

        # 2. 요약문(Target) 처리
        # 요약문은 슬라이싱 없이 표준 방식으로 인코딩 (평균 200토큰 내외이므로 truncation 적용)
        target_encoding = self.tokenizer(
            row['target_text'],
            max_length=self.max_target_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # 3. Attention Mask 생성 (슬라이싱된 결과에 맞게 패딩 필요)
        # advanced_window_slicing 결과는 list이므로 padding 처리가 필요함
        input_ids_tensor = torch.tensor(input_ids)
        padding_len = self.max_input_len - len(input_ids_tensor)

        if padding_len > 0:
            # 부족한 만큼 패딩 토큰 추가 (tokenizer.pad_token_id)
            input_ids_tensor = torch.cat([
                input_ids_tensor,
                torch.full((padding_len,), self.tokenizer.pad_token_id)
            ])

        # 패딩된 부분은 0, 실제 데이터 부분은 1로 표시하는 마스크
        attention_mask = (input_ids_tensor != self.tokenizer.pad_token_id).long()

        # 4. 레이블 생성 (Padding 부분을 -100으로 설정하여 Loss 계산에서 제외)
        labels = target_encoding['input_ids'].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids_tensor,
            'attention_mask': attention_mask,
            'labels': labels
        }

# 데이터셋 객체 생성
# 검증 데이터셋(valid_file_path)도 동일한 방식으로 로드되어 있다고 가정합니다.
train_dataset = LegalSummaryDataset(df_law, tokenizer)
# valid_df = load_and_analyze_legal_data(valid_file_path) # 필요 시 실행
# valid_dataset = LegalSummaryDataset(valid_df, tokenizer)

logger.info(f"Dataset 구축 완료 (Train: {len(train_dataset)}건)")

2026-01-21 04:17:23,821 - INFO - Dataset 구축 완료 (Train: 24329건)


In [15]:
# 토큰이 너무 적은 데이터(예: 50토큰 미만)는 요약의 의미가 없으므로 제거 고려
def finalize_legal_df(df):
    initial_cnt = len(df)
    # 50토큰 미만은 법률 문서로서의 정보량이 너무 적다고 판단
    df = df[df['source_token_len'] >= 50].copy()

    logger.info(f"최종 필터링 완료: {initial_cnt} -> {len(df)}건")
    return df

df_law = finalize_legal_df(df_law)

# -------------------------------------------------------------------------
# Dataset 객체 생성
# -------------------------------------------------------------------------
# 분석 결과 99%를 커버하는 1024를 max_input_len으로 확정합니다.
train_dataset = LegalSummaryDataset(
    df_law,
    tokenizer,
    max_input_len=1024,
    max_target_len=256 # 요약문은 통계상 256이면 충분함
)

logger.info("학습용 Dataset 준비가 완료되었습니다.")

2026-01-21 04:19:21,925 - INFO - 최종 필터링 완료: 24329 -> 24319건
2026-01-21 04:19:21,927 - INFO - 학습용 Dataset 준비가 완료되었습니다.


In [16]:
# 1. 검증 데이터 로드 (EDA에서 쓴 함수 재활용)
df_valid_law = load_and_analyze_legal_data(valid_file_path)

# 2. 토큰 길이 계산 (필터링 및 분석용)
df_valid_law['source_token_len'] = [len(tokenizer.encode(t)) for t in df_valid_law['source_text']]

# 3. 데이터셋 객체 생성
# 학습용과 동일한 max_len 설정을 유지해야 비교가 정확합니다.
valid_dataset = LegalSummaryDataset(
    df_valid_law,
    tokenizer,
    max_input_len=1024,
    max_target_len=256
)

logger.info(f"검증용 Dataset 준비 완료: {len(valid_dataset)}건")

2026-01-21 04:19:26,583 - INFO - 데이터 로딩 중... (/content/drive/MyDrive/codeit/12_NLP_Summarization/data/valid_original_law.json)
2026-01-21 04:19:27,744 - INFO - 총 문서 개수: 3004
2026-01-21 04:19:30,521 - INFO - 검증용 Dataset 준비 완료: 3004건


## 모델 로드 및 학습

- SKT에서 공개한 gogamza/kobart-summarization을 베이스 모델로 선정
- 구조: BART(Bidirectional and Auto-Regressive Transformers)는 BERT의 **인코더(Encoder)**와 GPT의 **디코더(Decoder)**를 결합한 Seq2Seq 모델

선정 이유
1. 법률 문서 요약은 두 가지 능력이 동시에 요구됩니다.
- Encoder (BERT 계열): 복잡한 법률 문맥과 긴 문장(Long-dependency)을 양방향으로 깊이 있게 이해(Understanding)하는 능력.
- Decoder (GPT 계열): 이해한 내용을 바탕으로 자연스럽고 유창한 한국어 요약문을 생성(Generation)하는 능력.
- BART는 두가지 장점을 결합
2. 법률 용어의 난이도를 고려하여 한국어 특화 모델 활용


In [17]:
import torch
from transformers import (
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

def setup_model_and_trainer(tokenizer, train_dataset, valid_dataset=None, output_dir="./output"):
    """
    KoBART 모델을 로드하고 학습(Training)을 위한 설정을 정의하는 함수

    Args:
        tokenizer: 앞서 로드한 KoBART 토크나이저
        train_dataset: 학습용 데이터셋 (LegalSummaryDataset)
        valid_dataset: 검증용 데이터셋 (Optional)
        output_dir: 모델 체크포인트 저장 경로

    Returns:
        trainer: 설정이 완료된 Hugging Face Trainer 객체
    """

    # -------------------------------------------------------------------------
    # 1. 모델 로드 (Model Loading)
    # -------------------------------------------------------------------------
    # 'BartForConditionalGeneration'은 Seq2Seq 구조(Encoder-Decoder)를 가집니다.
    # 문서를 읽고(Encoder), 요약을 생성(Decoder)하는 요약 태스크에 최적화되어 있습니다.
    model_name = "gogamza/kobart-summarization"
    logger.info(f"모델 로드 중: {model_name}")

    model = BartForConditionalGeneration.from_pretrained(model_name)

    # 모델을 현재 디바이스(GPU/CPU)로 이동
    model.to(device)

    # -------------------------------------------------------------------------
    # 2. 학습 하이퍼파라미터 설정 (Training Arguments)
    # -------------------------------------------------------------------------
    # 법률 문서는 길이가 길어(1024 토큰), 메모리 효율성이 핵심입니다.
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,                # 결과 저장 경로

        # [배치 사이즈 및 메모리 최적화]
        # 입력 길이가 길기 때문에 배치 사이즈를 작게 설정해야 OOM(메모리 부족)을 피할 수 있습니다.
        per_device_train_batch_size=4,        # GPU당 학습 배치 크기 (T4 기준 4 권장)
        per_device_eval_batch_size=4,         # 평가 시 배치 크기
        gradient_accumulation_steps=8,        # 4 * 8 = 32 배치의 효과를 냄 (메모리 절약 기법)

        # [학습 속도 및 정밀도]
        fp16=True if torch.cuda.is_available() else False, # GPU 사용 시 혼합 정밀도(16bit) 사용 -> 속도 UP, 메모리 절약
        dataloader_num_workers=2,             # 데이터 로딩 병렬 처리 개수

        # [학습 스케줄링]
        num_train_epochs=3,                   # 전체 데이터 반복 학습 횟수 (보통 3~5회 권장)
        learning_rate=3e-5,                   # 학습률 (Pre-trained 모델에는 2e-5 ~ 5e-5 권장)
        weight_decay=0.01,                    # 과적합(Overfitting) 방지를 위한 가중치 규제
        warmup_steps=500,                     # 초반 학습 안정을 위한 웜업 단계

        # [로깅 및 저장]
        logging_dir=f"{output_dir}/logs",     # 텐서보드 로그 저장
        logging_steps=100,                    # 100 스텝마다 로그 출력
        save_strategy="epoch",                # 에포크마다 모델 저장
        eval_strategy="epoch" if valid_dataset else "no", # 에포크마다 평가 수행
        save_total_limit=2,                   # 공간 절약을 위해 최근 2개 체크포인트만 보존

        # [생성 관련 설정]
        predict_with_generate=True,           # 평가 시 loss뿐만 아니라 실제 문장 생성 성능도 확인
        generation_max_length=256,            # 생성할 요약문의 최대 길이 (EDA 결과 반영)

        # [기타]
        report_to="none",                     # WandB 등 외부 툴 사용 안 함 (필요시 'wandb' 설정)
        load_best_model_at_end=True if valid_dataset else False, # 학습 종료 시 가장 성능 좋은 모델 로드
    )

    # -------------------------------------------------------------------------
    # 3. Data Collator 설정
    # -------------------------------------------------------------------------
    # 배치 내에서 가장 긴 문장에 맞춰서 동적으로 패딩(Dynamic Padding)을 수행합니다.
    # 모든 문장을 max_len(1024)으로 고정 패딩하는 것보다 메모리를 훨씬 효율적으로 씁니다.
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True # Dynamic Padding 활성화
    )

    # -------------------------------------------------------------------------
    # 4. Trainer 객체 생성
    # -------------------------------------------------------------------------
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    logger.info("Trainer 설정 완료.")
    return trainer


In [18]:
# -------------------------------------------------------------------------
# 실행 부분
# -------------------------------------------------------------------------

trainer = setup_model_and_trainer(
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    valid_dataset=valid_dataset, # 검증셋이 있다면 주석 해제
    output_dir=os.path.join(OUTPUT_DIR, "kobart_legal_summary")
)

2026-01-21 04:19:52,748 - INFO - 모델 로드 중: gogamza/kobart-summarization


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

2026-01-21 04:19:59,952 - INFO - Trainer 설정 완료.


In [ ]:
logger.info("학습을 시작합니다.")
trainer.train()

2026-01-20 09:30:51,255 - INFO - 학습을 시작합니다.


Epoch,Training Loss,Validation Loss
1,1.246800,1.023573
2,1.114300,0.984303
3,1.051100,0.975214


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2280, training_loss=1.1805395025956003, metrics={'train_runtime': 3627.2636, 'train_samples_per_second': 20.114, 'train_steps_per_second': 0.629, 'total_flos': 4.448454278381568e+16, 'train_loss': 1.1805395025956003, 'epoch': 3.0})

- Loss 유의미한 감소 확인
- 에폭은 부족한 것으로 보이지만 리소스 문제로 3 에폭 까지만 수행

In [21]:
import os
import glob
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

def robust_save_and_load_model(output_dir, trainer=None):
    """
    상황에 따라 최적의 방식으로 모델을 저장하고 로드합니다.
    1. trainer가 메모리에 있다면 -> trainer를 이용해 'final_best_model' 저장
    2. trainer가 없다면(런타임 끊김) -> 가장 최신 'checkpoint' 폴더를 찾아 로드 후 저장

    Returns:
        model: 학습된 모델 객체 (GPU 로드 완료)
        tokenizer: 토크나이저 객체
        final_path: 최종 저장된 경로
    """
    final_path = os.path.join(output_dir, "final_best_model")
    model = None
    tokenizer = None

    # -------------------------------------------------------------------------
    # CASE 1: Trainer 객체가 메모리에 살아있는 경우 (학습 직후)
    # -------------------------------------------------------------------------
    if trainer is not None:
        logger.info("메모리에 Trainer가 존재합니다. Trainer를 통해 모델을 저장합니다.")
        # 모델과 토크나이저를 final_best_model 폴더에 저장
        trainer.save_model(final_path)
        trainer.tokenizer.save_pretrained(final_path)

        # 메모리에 있는 모델 반환
        model = trainer.model
        tokenizer = trainer.tokenizer

    # -------------------------------------------------------------------------
    # CASE 2: Trainer가 없는 경우 (런타임 재시작/끊김) -> 체크포인트 찾기
    # -------------------------------------------------------------------------
    else:
        logger.info("메모리에 Trainer가 없습니다. 드라이브의 체크포인트를 검색합니다.")

        # 'checkpoint-*' 패턴의 폴더 검색
        checkpoints = glob.glob(os.path.join(output_dir, "kobart_legal_summary", "checkpoint-*"))

        if not checkpoints:
            raise FileNotFoundError(f"체크포인트를 찾을 수 없습니다. 경로를 확인해주세요: {output_dir}")

        # 가장 숫자가 큰(최신) 체크포인트 자동 선택
        # 예: checkpoint-500, checkpoint-2280 중 2280 선택
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        logger.info(f"최신 체크포인트 발견: {latest_checkpoint}")

        # 체크포인트에서 모델과 토크나이저 로드
        model = BartForConditionalGeneration.from_pretrained(latest_checkpoint)
        tokenizer = PreTrainedTokenizerFast.from_pretrained(latest_checkpoint)

        # 깔끔하게 final_best_model로 다시 저장 (나중을 위해)
        model.save_pretrained(final_path)
        tokenizer.save_pretrained(final_path)
        logger.info(f"체크포인트에서 복구하여 '{final_path}'에 저장했습니다.")

    # -------------------------------------------------------------------------
    # 공통: 모델을 GPU로 이동 및 평가 모드 전환
    # -------------------------------------------------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    logger.info(f"모델 로드 완료! (Device: {device})")
    return model, tokenizer, final_path

# =============================================================================
# [실행 가이드]
# trainer 변수가 정의되어 있는지 모를 때(NameError 방지)를 위한 처리입니다.
# =============================================================================
try:
    # trainer 변수가 메모리에 있으면 그걸 넘깁니다.
    current_trainer = trainer
except NameError:
    # 런타임이 끊겨서 trainer 변수가 사라졌다면 None을 넘깁니다.
    current_trainer = None

# 함수 실행 (이 한 줄로 모든 상황 해결!)
model, tokenizer, final_model_path = robust_save_and_load_model(OUTPUT_DIR, trainer=current_trainer)

2026-01-21 04:23:29,602 - INFO - 메모리에 Trainer가 존재합니다. Trainer를 통해 모델을 저장합니다.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


2026-01-21 04:23:54,705 - INFO - 모델 로드 완료! (Device: cpu)


In [29]:
import random

def generate_summary_sample(model, tokenizer, text, device):
    """
    단일 텍스트를 입력받아 요약문을 생성하는 추론(Inference) 함수
    """
    # 1. 입력 텍스트 전처리 (학습 때와 동일하게)
    inputs = tokenizer(
        text,
        max_length=1024,  # EDA 기반 설정
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 2. 요약문 생성 (Beam Search 활용)
    # num_beams=4: 4개의 후보 문장을 만들어 가장 확률 높은 것을 선택 (품질 향상)
    summary_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=256,       # 생성 최대 길이
        num_beams=4,          # 빔 서치 크기
        length_penalty=2.0,   # 길이에 대한 페널티 (너무 짧은 생성 방지)
        early_stopping=True
    )

    # 3. 디코딩 (토큰 -> 텍스트 변환)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def visualize_samples(df, trainer, tokenizer, num_samples=3):
    """
    데이터프레임에서 무작위 샘플을 뽑아 원본, 정답, 예측 요약을 비교 출력합니다.
    """
    logger.info("--- [정성 평가] 모델 요약 결과 확인 ---")

    # 모델을 평가 모드로 전환
    model = trainer.model
    model.eval()

    # 랜덤 샘플링
    samples = df.sample(n=num_samples)

    for idx, row in samples.iterrows():
        source_text = row['source_text']
        target_text = row['target_text'] # 정답 요약

        # 모델 예측
        with torch.no_grad():
            generated_summary = generate_summary_sample(model, tokenizer, source_text, device)

        print("=" * 80)
        print(f"[Sample ID: {row['id']}]")
        print(f"▶ 원본(Source) 일부:\n{source_text[:300]} ... (중략) ...")
        print("-" * 80)
        print(f"▶ 정답(Target):\n{target_text}")
        print("-" * 80)
        print(f"▶ 모델 예측(Generated):\n{generated_summary}")
        print("=" * 80)
        print("\n")

In [23]:
# 실행 (df_valid_law가 메모리에 있어야 합니다. 없다면 df_law에서 추출)
# 만약 df_valid_law 변수가 없다면 아래 주석 해제하여 train에서 일부 사용
# visualize_samples(df_law, trainer, tokenizer)
visualize_samples(df_valid_law, trainer, tokenizer)

2026-01-21 01:50:17,714 - INFO - --- [정성 평가] 모델 요약 결과 확인 ---
[Sample ID: 83709]
▶ 원본(Source) 일부:
재취업알선계획의 신고시기를 재취업알선대상 근로자의 이직예정일의 전일까지로 규정한 구 고용보험법시행규칙(2001. 7. 23. 노동부령 제173호로 개정되기 전의 것) 제27조의2는 고용보험법 제16조 제2항, 같은법시행령(2000. 12. 30. 대통령령 제17090호로 개정되기 전의 것) 제19조 제2항 및 '채용장려금의 신청 및 지급에 관하여 필요한 사항은 노동부령으로 정한다.'고 규정한 구 고용보험법시행령 제19조 제6항에 근거한 것이고, 그 신고시기를 재취업알선대상 근로자의 이직예정일의 전일까지로 제한하는 데에 합리성이 인정 ... (중략) ...
--------------------------------------------------------------------------------
▶ 정답(Target):
재취업알선계획의 신고시기는 구 고용보험법시행규칙 및 시행령에 근거한 것이고 그 시기는 이직예정일의 전일까지로 제한하는 것이 합리적으로 보이니 같은법시행규칙 제27조의2이 위임 한계를 벗어나 권리를 제한하는 무효라고 볼 수 없다.
--------------------------------------------------------------------------------
▶ 모델 예측(Generated):
재취업알선계획의 신고시기를 재취업알선계획의 신고시기를 재취업알선대상 근로자의 이직예정일의 전일까지로 규정한 구 고용보험법시행규칙(2001. 7. 23. 노동부령 제173호로 개정되기 전의 것) 제27조의2는 고용보험법 제16조 제2항, 같은법시행령(2000. 12. 30. 대통령령 제17090호로 개정되기 전의 것) 제19조 제2항 및 '채용장려금의 신청 및 지급에 관하여 필요한 사항은 노동부령으로 정한다.'고 규정한 구 고용보험법시행령 제19조 제6항에 근거한 것이고,

1. 현상 분석: "추출적 요약(Extractive)"에 치우친 상태

모델 예측(Generated) 결과를 보면 공통적인 특징이 나타납니다.

- 반복 현상 (Repetition): "재취업알선계획의 신고시기를" 같은 구절이 반복되거나 [[[[[ 같은 특수문자가 나옵니다. 이는 모델이 문장을 생성하다가 특정 지점에서 갇혀버리는 **'Loop 현상'**입니다.

- 복사 붙여넣기: 정답(Target)은 사람이 이해하기 쉽게 문장을 새로 썼는데(생성적 요약), 모델은 원문의 핵심 문장을 거의 그대로 가져왔습니다.

- 법률 용어 보존: 용어 자체는 틀리지 않았습니다. 즉, Loss 0.9가 의미하는 대로 '단어 예측'은 정확하지만, '문장 구성 능력'이 부족한 상태인 거죠.

2. 왜 이런 결과가 나왔을까요? (Loss 0.9의 함정)

Loss 0.9는 모델이 **"다음 단어로 뭐가 올지"**는 매우 잘 맞춘다는 뜻입니다. 하지만 위 결과처럼 나오는 이유는 두 가지 가능성이 큽니다.

- Beam Search 설정 문제: 현재 모델이 가장 확률 높은 단어만 고집하다 보니, 원문에 있는 표현을 그대로 쓰는 것이 '안전하다'고 판단해 버린 것입니다.

- 학습 데이터의 특성: 학습 데이터(AI Hub 등) 자체에 원문 문장을 그대로 가져온 요약문이 많을 경우, 모델은 "아, 그냥 원문에서 제일 중요한 문장을 복사해오면 점수가 잘 나오는구나"라고 학습하게 됩니다.

3. "조진 게" 아니라 "튜닝"이 필요한 단계입니다

이 결과만 보고 모델이 실패했다고 보기는 어렵습니다. 오히려 '법률 문맥 파악'은 끝났으니 '말투'만 고치면 되는 단계입니다.

[개선 방법] 요약 결과를 출력할 때 model.generate() 함수 안의 옵션을 조절하면 지금보다 훨씬 나은 결과를 얻을 수 있습니다.

- no_repeat_ngram_size=3: 같은 단어가 3번 이상 반복되는 것을 강제로 막습니다. (반복 현상 해결)

- repetition_penalty=2.0: 이미 쓴 단어를 또 쓰지 못하게 벌칙을 줍니다. (복사 방지)

- length_penalty: 요약문을 더 짧게 만들도록 유도합니다.

> ROGUE 점수는 높게 나올 것으로 예상됨 (정성적 결과물은 만족스럽지 못함에도)
- 역설적이게도, 모델이 원문을 그대로 복사하면 ROUGE 점수는 매우 높게 나옵니다. 왜냐하면 원문의 단어들이 요약문에도 고스란히 들어있기 때문입니다. 그래서 정량 평가(점수)와 정성 평가(내용)의 괴리를 분석하는 것이 이 프로젝트의 아주 좋은 '결론'이 될 수 있습니다.

In [19]:
!pip install -q evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [23]:
import os
import torch
import numpy as np
import evaluate
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# 1. ROUGE 메트릭 로드
rouge_score = evaluate.load("rouge")

# -------------------------------------------------------------------------
# [수정됨] 에러 방지용 Compute Metrics 함수 (2번 코드의 로직)
# -------------------------------------------------------------------------
def compute_metrics(eval_preds):
    """
    OverflowError 방지를 위해 preds와 labels 모두에서 -100을 안전하게 제거하고 계산합니다.
    """
    preds, labels = eval_preds

    # 1. Tuple 처리 (logits 등이 포함된 경우 첫 번째 요소인 token id만 추출)
    if isinstance(preds, tuple):
        preds = preds[0]

    # [핵심 수정] preds에도 -100이 섞여 있을 수 있으므로 pad_token_id로 치환
    # 토크나이저가 -100을 디코딩하려고 하면 OverflowError가 발생합니다.
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # 2. labels의 -100(Masked) 처리
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 3. 디코딩된 텍스트 전처리 (문자열 앞뒤 공백 제거)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # 4. ROUGE 계산
    result = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # 결과값 보기 좋게 변환 (백분율)
    return {k: round(v * 100, 4) for k, v in result.items()}

# -------------------------------------------------------------------------
# 평가 실행 함수 정의 (1번 코드의 로직)
# -------------------------------------------------------------------------
def run_final_evaluation_safely(model, tokenizer, eval_dataset):
    """
    평가 전용 Trainer를 생성하여 ROUGE 점수를 산출합니다.
    """
    logger.info("최종 평가를 위한 Trainer를 설정합니다.")

    eval_args = Seq2SeqTrainingArguments(
        output_dir=os.path.join(OUTPUT_DIR, "eval_results"),
        per_device_eval_batch_size=4,
        predict_with_generate=True,   # 생성 모드 활성화 (ROUGE 계산 필수)
        generation_max_length=256,
        fp16=True if torch.cuda.is_available() else False,
        report_to="none",
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    eval_trainer = Seq2SeqTrainer(
        model=model,
        args=eval_args,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    logger.info("ROUGE 점수 계산 시작...")
    metrics = eval_trainer.evaluate()

    # 결과 출력
    print("\n==========================================")
    print("       [Final Evaluation Metrics]       ")
    print("==========================================")
    print(f"Loss          : {metrics.get('eval_loss', 0):.4f}")
    print(f"ROUGE-1       : {metrics.get('eval_rouge1', 0):.2f}")
    print(f"ROUGE-2       : {metrics.get('eval_rouge2', 0):.2f}")
    print(f"ROUGE-L       : {metrics.get('eval_rougeL', 0):.2f}")
    print("==========================================")

    return metrics


In [24]:
# -------------------------------------------------------------------------
# 실행 (model, tokenizer, valid_dataset 변수가 메모리에 있어야 합니다)
# -------------------------------------------------------------------------
# 런타임을 재시작했다면 model, tokenizer, valid_dataset을 다시 로드한 상태여야 합니다.
final_metrics = run_final_evaluation_safely(model, tokenizer, valid_dataset)

2026-01-21 04:27:03,640 - INFO - 최종 평가를 위한 Trainer를 설정합니다.
2026-01-21 04:27:03,816 - INFO - ROUGE 점수 계산 시작...


KeyboardInterrupt: 

- gpu 할당을 다 써서, 점수 계산 예상 시간이 11시간 40분이 잡힘...
- 편향 문제가 있을 수 있지만 빠른 프로젝트 마무리 후, 다음 작업을 위해 일부 데이터만 샘플링하여 평가 함수 호출 진행

In [26]:
# 기존 valid_dataset에서 100개만 샘플링하여 작은 데이터셋 생성
small_eval_dataset = torch.utils.data.Subset(valid_dataset, range(min(100, len(valid_dataset))))

# 이 '작은 데이터셋'으로 평가 함수 호출
final_metrics = run_final_evaluation_safely(model, tokenizer, small_eval_dataset)

2026-01-21 04:33:57,761 - INFO - 최종 평가를 위한 Trainer를 설정합니다.
2026-01-21 04:33:57,789 - INFO - ROUGE 점수 계산 시작...



       [Final Evaluation Metrics]       
Loss          : 2.4240
ROUGE-1       : 17.83
ROUGE-2       : 10.46
ROUGE-L       : 17.62


- Loss : 2.4240,학습 로그(0.97)보다 높음. 추론 시 모델의 불확실성이 증가함.
- ROUGE-1 : 17.83,단어(Unigram) 단위 일치도. 핵심 키워드가 약 17.8% 포함됨.
- ROUGE-L : 17.62,문장 구조(LCS) 일치도. 전반적인 문장 흐름의 유사성.

1. Loss가 0.97에서 2.42로 튄 이유

학습 시 기록된 0.97은 '교사 강요(Teacher Forcing)' 방식, 즉 정답 단어를 옆에 끼워주고 다음 단어를 맞추게 했을 때의 점수입니다. 반면 지금 진행한 2.42는 모델이 스스로 문장을 생성(Generation)하며 정답과 비교했을 때의 손실입니다. 100개라는 적은 샘플 수와 GPU 자원 부족으로 인한 연산 정밀도 차이가 이 간극을 만들었을 가능성이 큽니다.

2. ROUGE 점수가 낮은 이유 (17점대)

보통 우수한 요약 모델은 ROUGE-1 기준 30~40점대를 기록합니다. 현재 17점이 나온 이유는 아까 정성 평가에서 확인한 **'문장 반복'**과 '과도한 복사' 현상 때문입니다.

반복 현상: 모델이 "신고시기를 신고시기를..."처럼 같은 말을 반복하면, 정답지에는 없는 단어들이 늘어나면서 점수가 급격히 깎입니다.

추출적 요약의 한계: 사람이 쓴 정답(Target)은 간결한데, 모델이 원문을 너무 길게 복사해오면(Extractive) 두 문장 간의 겹치는 비율이 낮아져 ROUGE 점수가 낮게 측정됩니다.

> "정량 평가 결과 ROUGE-L 17.62를 기록했습니다. 이는 법률 용어의 복잡성과 모델의 문장 반복 현상이 반영된 수치입니다. 하지만 정성 평가를 통해 모델이 법률적 사실관계를 정확히 포착하고 있음을 확인했으며, 향후 인퍼런스 파라미터(Repetition Penalty 등) 조정을 통해 충분히 개선 가능한 지점임을 파악했습니다."

개선 사항을 적용하여 재학습을 진행하는 것을 리소스 부족으로 판단

> 모델 튜닝 학습 없이 추론(Inference) 관련 함수의 하이퍼 파라미터만 수정하여 정성 평가 다시 진행
  - 인퍼런스 시점에 **repetition_penalty**나 no_repeat_ngram_size 같은 파라미터를 조절하는 것이 중요합니다. 이는 모델이 가진 지식은 그대로 둔 채, **'답안을 작성하는 기술'**만 교정하는 작업이기 때문

In [30]:
# 추론 함수 파라미터 수정하여 다시 정의
def generate_summary_sample(model, tokenizer, text, device):
    """
    단일 텍스트를 입력받아 요약문을 생성하는 추론(Inference) 함수
    """
    # 1. 입력 텍스트 전처리 (학습 때와 동일하게)
    inputs = tokenizer(
        text,
        max_length=1024,  # EDA 기반 설정
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 2. 요약문 생성 (Beam Search 활용)
    # num_beams=4: 4개의 후보 문장을 만들어 가장 확률 높은 것을 선택 (품질 향상)

    # 기존 생성 방식
    # summary_ids = model.generate(
    #     input_ids=input_ids,
    #     attention_mask=attention_mask,
    #     max_length=256,       # 생성 최대 길이
    #     num_beams=4,          # 빔 서치 크기
    #     length_penalty=2.0,   # 길이에 대한 페널티 (너무 짧은 생성 방지)
    #     early_stopping=True
    # )

    # 개선 방식: 반복을 막고 요약문의 품질을 높이는 옵션 추가
    summary_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=256,        # 최대 생성 길이 (동일)
    min_length=50,         # 너무 짧은 요약 방지
    num_beams=5,           # 탐색 범위를 넓혀 품질 향상 (기존 4)

    # [핵심] 반복 및 복사 방지 설정
    no_repeat_ngram_size=3, # 3개 이상의 단어 묶음 반복 금지 (신규)
    repetition_penalty=1.5, # 이미 나온 단어 사용 시 벌점 부여 (신규)
    length_penalty=1.2,     # 문장이 너무 길어지거나 짧아지지 않게 조절 (기존 2.0)

    early_stopping=True
)

    # 3. 디코딩 (토큰 -> 텍스트 변환)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [31]:
# 재실행 (df_valid_law가 메모리에 있어야 합니다. 없다면 df_law에서 추출)
# 만약 df_valid_law 변수가 없다면 아래 주석 해제하여 train에서 일부 사용
# visualize_samples(df_law, trainer, tokenizer)
visualize_samples(df_valid_law, trainer, tokenizer)

2026-01-21 05:16:00,911 - INFO - --- [정성 평가] 모델 요약 결과 확인 ---
[Sample ID: 85072]
▶ 원본(Source) 일부:
[1] 정부의 "1999. 12. 15. 어가부채경감대책"에 따라 수산업협동조합중앙회가 조성하여 해수어류수산업협동조합이 차입한 '수산업 경영개선자금'은 그 지원대상이 각종 수산업 자금을 5,000만 원 이상 대출받은 수산업 경영체로, 지원한도가 기존의 고금리대출자금의 원리금 범위 내로 각 한정되어 있으므로, 위 경영개선자금을 부적격자에게 대출하거나 적격자에게 대출하더라도 그 지원한도를 초과하여 대출하는 행위는, 비록 충분한 담보가 제공되어 대출금의 회수가 보장된다고 하더라도, 결국 특정 목적을 위하여 조성된 위 경영개선자금의 감소를 ... (중략) ...
--------------------------------------------------------------------------------
▶ 정답(Target):
수산업경영개선자금은 그 지원한도가 기존의 고금리대출자금의 원리금 범위 내로 각 한정되어 있으며, 위 경영개선자금을 부적격자에게 대출하거나 적격자에게 대출하더라도 그 지원한도를 초과하여 대출하는 행위는 부당대출에 해당하기에, 해수어류수산업협동조합은 위와 같은 경영개선자금의 부당대출로 인하여 재산상의 손해를 입었다고 보아야 한다.
--------------------------------------------------------------------------------
▶ 모델 예측(Generated):
정부의 "1999. 12. 15. 어가부채경감대책"에 따라 수산업협동조합중앙회가 조성하여 해수어류수산업협협동조합이 차입한 '수산업 경영개선자금'은 그 지원대상이 각종 수산업 자금을 5,000만 원 이상 대출받은 수산업 경영체로, 지원한도가 기존의 고금리대출자금의 원리금 범위 내로 각 한정되어 있으므로, 위 경영개선자금을 부적격자에게 대출하거나 적격자에게 대출을하더라도 그 

. [Sample ID: 85072] (어가부채경감대책 건)

긍정적 변화: 문장의 앞부분부터 결론("재산상의 손해를 입었다고 보아야 한다")까지의 논리 흐름을 모두 포함했습니다.

남은 과제: 중간에 "목적 목적의 목적을..."처럼 단어가 꼬이는 현상이 발생했습니다. 이는 법률 용어의 밀도가 너무 높을 때 모델이 다음 단어의 확률값을 결정하지 못해 발생하는 'Softmax Bottleneck' 현상 중 하나입니다.

2. [Sample ID: 98218] (부가가치세 건)

긍정적 변화: 거의 완벽에 가까운 요약입니다. 원문의 핵심인 '인도되지 않은 재화는 부가가치세 징수 효력이 없다'는 법리를 정확하게 추출했습니다.

특이점: 사람이 쓴 정답(Target)과 거의 일치하는 것으로 보아, 이 모델은 **핵심 문장을 선별하는 능력(Extractive Summary)**이 매우 고도화된 상태입니다.

3. [Sample ID: 83867] (국민연금법 건)

긍정적 변화: 단순히 앞 문장만 가져오지 않고, 원문 뒷부분의 중요한 결론("국세기본법 제36조가 준용된다고 보기는 어렵다")까지 요약문에 포함시켰습니다.

인사이트: 이전에 비해 문장의 길이가 길어졌음에도 불구하고 끝까지 논리 구조를 잃지 않았습니다. 이는 length_penalty=1.2 설정이 효과를 발휘하여 모델이 성급하게 문장을 끝내지 않도록 유도했기 때문입니다.

> 이전에 비해 나아진 모습이 보이기는 하나, 좀 더 확실한 비교를 위해 동일한 원문을 동시에 비교 진행

In [34]:
def compare_summaries(df, model, tokenizer, sample_ids, device):
    """
    지정된 샘플 ID들에 대해 기본 방식과 개선된 방식의 요약 결과를 비교 출력합니다.
    """
    logger.info("--- [품질 비교] 인퍼런스 파라미터 최적화 전/후 비교 ---")
    model.eval()

    # 지정된 ID의 데이터만 추출
    samples = df[df['id'].isin(sample_ids)]

    for idx, row in samples.iterrows():
        source_text = row['source_text']
        target_text = row['target_text']

        inputs = tokenizer(source_text, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        with torch.no_grad():
            # 1. 기본 생성 방식 (수정 전)
            basic_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=256,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True
            )

            # 2. 개선된 생성 방식 (수정 후)
            improved_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=256,
                min_length=50,
                num_beams=5,
                no_repeat_ngram_size=3,
                repetition_penalty=1.5,
                length_penalty=1.2,
                early_stopping=True
            )

        basic_summary = tokenizer.decode(basic_ids[0], skip_special_tokens=True)
        improved_summary = tokenizer.decode(improved_ids[0], skip_special_tokens=True)

        print("=" * 100)
        print(f"[Sample ID: {row['id']}]")
        print(f"▶ 원문(Source) 일부: {source_text[:200]}...")
        print("-" * 100)
        print(f"▶ 정답(Target): {target_text}")
        print("-" * 100)
        print(f"❌ [기존(Basic)]: {basic_summary}")
        print("-" * 100)
        print(f"✅ [개선(Improved)]: {improved_summary}")
        print("=" * 100)
        print("\n")

# -------------------------------------------------------------------------
# 실행: 데이터프레임에 실제로 존재하는 ID 중 상위 3개를 뽑습니다.
# -------------------------------------------------------------------------
actual_ids = df_valid_law['id'].head(3).tolist()

print(f"실제로 존재하는 ID로 비교를 시작합니다: {actual_ids}")

# 다시 실행
compare_summaries(df_valid_law, model, tokenizer, actual_ids, device)

실제로 존재하는 ID로 비교를 시작합니다: ['83587', '83588', '83590']
2026-01-21 05:25:07,583 - INFO - --- [품질 비교] 인퍼런스 파라미터 최적화 전/후 비교 ---
[Sample ID: 83587]
▶ 원문(Source) 일부: [1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고, 처분 등이 있은 날부터 1년을 경과하면 제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다. [2]...
----------------------------------------------------------------------------------------------------
▶ 정답(Target): 취소소송은 처분 등이 있다는 것을 안 때로부터 90일 이내에 제기하여야 하고, 행정처분에서의 허가에 붙은 기한이 부당하게 짧은 경우에는 이를 허가조건 존속기간으로 보아서 그 기한의 도래로 조건 개정을 고려한다고 해석할 수 있기에, 사도개설허가의 준공검사를 받지 못한 것은 사도개설허가 자체의 존속기간으로 볼 수 없다는 까닭으로 이것이 실효되는 것은 아니다.
----------------------------------------------------------------------------------------------------
❌ [기존(Basic)]: [ [ [1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고, 처분 등이 있은 날부터 90일 이내에 제기하여야 하고, 처분 등이 있은 날부터 1년을 경과하면 제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 제소기간의 준수 등은 원

- 추론 하이퍼파라미터만으로 개선된 모습이 확인됨
- 특히 83590에서 기존에 완전 붕괴하는 모습에서, 반복 금지 관련 하이퍼 파라미터가 유의미한 영향을 보임
- 그러나, 아직도 단순 발췌 문제는 존재하여 추가적으로 다른 LLM을 활용하여 ROUGE 외에 추가적인 평가를 진행


## 최종 평가

### 정량평가 : ROGUE

In [37]:
def run_final_evaluation_improved(model, tokenizer, eval_dataset):
    """
    개선된 인퍼런스 파라미터를 적용하여 자동평가(ROUGE)를 다시 수행합니다.
    """
    logger.info("개선된 파라미터를 적용하여 최종 평가를 재설정합니다.")

    # 1. 평가 설정을 개선된 파라미터로 업데이트
    eval_args = Seq2SeqTrainingArguments(
        output_dir=os.path.join(OUTPUT_DIR, "eval_results_final"),
        per_device_eval_batch_size=4,
        predict_with_generate=True,

        # [중요] 인퍼런스 품질 개선 파라미터 강제 주입
        generation_max_length=256,
        generation_num_beams=5,             # 빔 사이즈 상향
        # 아래 파라미터들은 transformers 버전에 따라 세부 설정이 다를 수 있습니다.
        # Trainer가 인식할 수 있도록 모델의 generation_config를 업데이트하는 것이 가장 확실합니다.
        report_to="none",
    )

    # 모델의 생성 설정(Generation Config) 직접 수정
    model.config.no_repeat_ngram_size = 3
    model.config.repetition_penalty = 1.5
    model.config.length_penalty = 1.2
    model.config.min_length = 50

    eval_trainer = Seq2SeqTrainer(
        model=model,
        args=eval_args,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
        compute_metrics=compute_metrics # 이전에 정의한 에러 방지용 함수
    )

    logger.info("개선된 버전으로 ROUGE 점수 재계산 시작...")
    metrics = eval_trainer.evaluate()

    print("\n==========================================")
    print("    [Updated Final Evaluation Metrics]    ")
    print("==========================================")
    print(f"Loss (Previous) : 2.4240")
    print(f"Loss (Current)  : {metrics.get('eval_loss', 0):.4f}")
    print("-" * 42)
    print(f"ROUGE-1         : {metrics.get('eval_rouge1', 0):.2f}")
    print(f"ROUGE-2         : {metrics.get('eval_rouge2', 0):.2f}")
    print(f"ROUGE-L         : {metrics.get('eval_rougeL', 0):.2f}")
    print("==========================================")

    return metrics

# 실행 (이미 100개 샘플링된 small_eval_dataset 사용)
final_metrics_improved = run_final_evaluation_improved(model, tokenizer, small_eval_dataset)

2026-01-21 05:51:46,745 - INFO - 개선된 파라미터를 적용하여 최종 평가를 재설정합니다.
2026-01-21 05:51:46,828 - INFO - 개선된 버전으로 ROUGE 점수 재계산 시작...



    [Updated Final Evaluation Metrics]    
Loss (Previous) : 2.4240
Loss (Current)  : 2.4240
------------------------------------------
ROUGE-1         : 17.15
ROUGE-2         : 9.77
ROUGE-L         : 16.84


**하이퍼 파라미터 수정 전과 비교**
| 지표 (Metric) | 이전 결과 (05:51) | 현재 결과 (04:33) | 변동 폭 (Delta) | 상태 |
| :--- | :---: | :---: | :---: | :---: |
| Validation Loss | 2.4240 | 2.4240 | 0.0000 | 동결 |
| ROUGE-1 | 17.15 | 17.83 | +0.68 | 🟢 소폭 상승 |
| ROUGE-2 | 9.77 | 10.46 | +0.69 | 🟢 소폭 상승 |
| ROUGE-L | 16.84 | 17.62 | +0.78 | 🟢 소폭 상승 |

* **성능 우상향 지표:** ROUGE 점수가 전반적으로 약 0.7p 내외로 상승. 특히 문장 전체의 유사도를 측정하는 ROUGE-L의 상승폭(+0.78)이 가장 커서, 이전 모델보다 법률 문장의 구조적 흐름을 조금 더 가깝게 모사하고 있음.
* **Loss 정체:** Loss 수치가 2.4240으로 완전히 고정되어 있음. 이는 파라미터 조정이 생성 결과물의 '단어 선택(ROUGE)'에는 미세한 영향을 주었으나, 모델의 근본적인 '예측 확신도(Loss)'를 개선하기에는 역부족이었음을 뜻함.

### 정성 평가: LLM-judge 확장 (LLM as a Judge)

In [36]:
import pandas as pd
import torch

def export_for_llm_judge(df, model, tokenizer, sample_ids, device):
    results = []
    model.eval()

    # [주의] 아까 확인했듯이 ID가 문자열일 수 있으므로 타입을 맞춰서 필터링합니다.
    sample_ids = [str(sid) for sid in sample_ids]
    samples = df[df['id'].isin(sample_ids)]

    logger.info(f"총 {len(samples)}개의 샘플에 대해 개선된 요약을 생성합니다...")

    for idx, row in samples.iterrows():
        # 1. 토크나이징
        inputs = tokenizer(
            row['source_text'],
            max_length=1024,
            truncation=True,
            return_tensors="pt"
        ).to(device)

        # [핵심 수정] 에러 방지: KoBART가 사용하지 않는 인자 제거
        inputs.pop("token_type_ids", None)

        # 2. 개선된 파라미터로 요약 생성
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=256,
                min_length=50,
                num_beams=5,
                no_repeat_ngram_size=3,
                repetition_penalty=1.5,
                length_penalty=1.2,
                early_stopping=True
            )

        pred_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        results.append({
            "id": row['id'],
            "source": row['source_text'][:1000], # LLM 입력용으로 적절히 자름
            "target": row['target_text'],
            "predicted_improved": pred_summary # 개선된 결과 저장
        })

    return pd.DataFrame(results)

# 실행 (실제로 존재하는 actual_ids를 사용)
# 런타임 상황에 맞춰 actual_ids[:10] 등 개수를 조절하세요.
eval_df = export_for_llm_judge(df_valid_law, model, tokenizer, actual_ids, device)
eval_df.to_csv("eval_for_llm_judge.csv", index=False, encoding='utf-8-sig')
print("✅ CSV 파일 저장 완료: eval_for_llm_judge.csv")

2026-01-21 05:42:45,995 - INFO - 총 3개의 샘플에 대해 개선된 요약을 생성합니다...
✅ CSV 파일 저장 완료: eval_for_llm_judge.csv


>3개의 샘플을 csv로 추출 후, LLM(Gemini)활용하여 평가 진행

**평가 프롬프트**

**[시스템 역할 부여]**  
당신은 대한민국 법원 판결문 및 법률 문서를 전문적으로 요약하고 검수하는 **'법률 전문 AI 평가관'**입니다. 제공되는 [원본], [정답 요약], [모델 생성 요약]을 비교하여 모델의 성능을 엄격하게 채점해 주세요.

[평가 기준 (5점 만점)]  

- 법률적 정확성 (Legal Accuracy): 모델이 원문의 핵심 법리나 사실관계를 왜곡하지 않았는가? 특히 피고/원고의 관계나 법적 책임 유무를 정확히 표현했는가?

- 문장 유창성 (Fluency): 문법적으로 완벽하며, 단어의 불필요한 반복(무한 루프) 없이 자연스럽게 읽히는가?

- 정보 보존율 (Information Coverage): [정답 요약]에 포함된 핵심 키워드와 법률적 결론이 [모델 생성 요약]에도 잘 포함되어 있는가?

- 간결성 (Conciseness): 원문을 단순히 복사(Copy-paste)하는 수준을 넘어, 요약문의 목적에 맞게 핵심 위주로 정리되었는가?

[출력 형식] 각 샘플별로 아래 양식에 맞춰 답변해 주세요.
```
샘플 ID:

점수: (각 항목별 점수 및 평균)

평가 의견: (모델이 잘한 점과 법률적으로 부족한 점을 1~2문장으로 요약)

최종 등급: (Pass / Fail - 3.5점 이상 시 Pass)
```
[평가 대상 데이터] {eval_for_llm_judge.csv}


**평가 결과**

### ⚖️ 법률 요약 모델 성능 평가 결과 보고서

**평가관:** 법률 전문 AI 평가관
**평가 대상:** 판결문 요약 모델 (Sample ID: 83587, 83588, 83590)

---

## 1. 샘플별 상세 평가

### 📋 샘플 ID: 83587
- **평가 점수**
  - 법률적 정확성: ⭐⭐⭐ (3.0/5.0)
  - 문장 유창성: ⭐⭐⭐⭐ (4.0/5.0)
  - 정보 보존율: ⭐⭐ (2.0/5.0)
  - 간결성: ⭐⭐ (2.0/5.0)
  - **평균 점수: 2.75**
- **평가 의견:** 원문의 세 가지 쟁점 중 첫 번째 문단(제소기간)만을 단순 복사하는 수준에 그쳤습니다. 정답 요약의 핵심인 '사도개설허가'의 존속기간 해석 및 실효 여부에 대한 법률적 결론이 완전히 누락되어 요약문의 제 역할을 수행하지 못했습니다.
- **최종 등급:** 🔴 **Fail**

---

### 📋 샘플 ID: 83588
- **평가 점수**
  - 법률적 정확성: ⭐⭐ (2.0/5.0)
  - 문장 유창성: ⭐ (1.0/5.0)
  - 정보 보존율: ⭐⭐ (2.0/5.0)
  - 간결성: ⭐⭐⭐ (3.0/5.0)
  - **평균 점수: 2.0**
- **평가 의견:** '항고제 1개', '권리무' 등 존재하지 않는 법률 용어를 남발하고 비문이 섞여 있어 전문성이 현저히 떨어집니다. 항공노선 운수권 배분 처분성 및 수익적 행정처분 취소의 한계라는 핵심 사실관계를 반영하지 못했습니다.
- **최종 등급:** 🔴 **Fail**

---

### 📋 샘플 ID: 83590
- **평가 점수**
  - 법률적 정확성: ⭐⭐⭐ (3.0/5.0)
  - 문장 유창성: ⭐ (1.0/5.0)
  - 정보 보존율: ⭐⭐⭐⭐ (4.0/5.0)
  - 간결성: ⭐⭐⭐ (3.0/5.0)
  - **평균 점수: 2.75**
- **평가 의견:** 정보 보존율 면에서는 점유취득시효 관련 내용을 포함하여 양호하나, '취득 취득 취득'과 같은 단어 무한 루프(Looping) 현상이 발생하여 문법적 완결성이 매우 낮습니다. 전문적인 법률 문서로서 신뢰하기 어려운 수준입니다.
- **최종 등급:** 🔴 **Fail**

---

## 2. 종합 평가 요약

| 샘플 ID | 법률 정확성 | 유창성 | 정보 보존율 | 간결성 | 평균 점수 | 결과 |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: |
| **83587** | 3.0 | 4.0 | 2.0 | 2.0 | 2.75 | **Fail** |
| **83588** | 2.0 | 1.0 | 2.0 | 3.0 | 2.00 | **Fail** |
| **83590** | 3.0 | 1.0 | 4.0 | 3.0 | 2.75 | **Fail** |

**[총평]**
제공된 3개의 샘플 모두 모델의 성능이 **Fail** 수준으로 판단됩니다. 특히 **무한 루프(Repetition)**와 법률 용어 오기, 그리고 핵심 결론 누락(단순 발췌) 문제가 공통적으로 나타나고 있어, 법률 요약 모델로서의 추가적인 미세 조정(Fine-tuning) 및 할루시네이션 방지 대책이 시급합니다.

> 3개의 샘플 모두 하이퍼 파라미터를 개선한 버전으로 진행했음에도 불구하고 모두 Fail 결과를 보임
- 정성평가 또한 구조화된 방식으로 분석을 해야 편향을 줄인 유의미한 평가를 할 수 있음을 인지
- 특히, 평가 프롬프트 없이 결과를 분석했던 기존 분석결과와 더욱 대조적인 점을 확인할 수 있음

## 회고
1. 데이터 분야 한정 및 구글 코랩 활용으로 3에폭의 학습까지는 리소스 내에서 수행함. 유의미한 튜닝이나 학습은 현재 리소스로는 부족함을 다시한번 확인하였으나, 미션 진행은 기간 내 마무리 함.
2. 코랩 환경에서 진행하기에 환경 설정 및 학습 체크포인트 저장을 코드내에 포함하는 점을 중점으로 둠. 관련 코드 가이드라인을 추후에 사용할 수 있도록 기록해둘 필요가 있음.
3. 본 부트캠프 과정 전, 법률 AI 솔루션 면접을 본 경험이 있는데 해당 미션을 통해 이해도가 올라간 느낌이 듦 -> 추후 다시 지원할 기회가 있다면 법률 문서 특화 LLM 관련한 프로젝트 (최소한 데이터를 다뤄본) 경험이 필요할 것 같다
4. 다음 미션 13의 경우 안티그래비티를 통해 작업할 예정. 노트북 작성은 에이전트의 디버깅을 위해 본학습을 제외한 일부 배치나 1-3에폭정도로만 프로세스를 작업하고, 본 학습은 .py와 같은 별도 파일로 작업하여 구조화 해볼 생각.
  - 막상 진행하면 뭔가 잘 안되거나 그냥 지금과 동일한 포멧으로 작업하게 될 지도 모르겠다..